# Optimizer Methods Testing

This is notebook for testing the workflow of the optimizer object in MDSAPT. Optimizer was developed to solve the issue of residues having an unbalanced spin when drawn directly from protein backbone. It does this over a few steps addressing the different issues. 

In [1]:
import os

import numpy as np

import MDAnalysis as mda

from pdbfixer import PDBFixer

from simtk.openmm.app import PDBFile

import nglview as nv

from MDSAPT.mdsapt.reader import InputReader

2021-12-28 14:13:57,350 MDSAPT       INFO     MDSAPT 0+untagged.80.g22967c3 starting
2021-12-28 14:13:57,353 MDSAPT       INFO     Copyright (c) 2021 Alia Lescoulie
2021-12-28 14:13:57,353 MDSAPT       INFO     Released under MIT Licence


In [7]:
In = InputReader('test_input.yaml')

unv = mda.Universe(In.top_path, In.trj_path)

'ncpu' Error in trajectory settings


## 1. Loading the Residue

In this case ARG is the residue. Both the amino and carboxyl side of the residue are missing protons.

In [8]:
res = unv.select_atoms('resid 2')

nv.show_mdanalysis(res)

NGLWidget(max_frame=97)

## 2. Replacing missing Amino Protons
The file is loaded into PDBFixer from opennm and amino protons are reattached. This is the `_fix_amino()` method in Optimizer.

In [9]:
res.write('resid.pdb', file_format='PDB') # Saving residue
fixer = PDBFixer(filename='resid.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingHydrogens(7) # Adding protons at pH value
PDBFile.writeFile(fixer.topology, fixer.positions, open('resid_fixed.pdb', 'w'))


res_fixed = mda.Universe('resid_fixed.pdb')
resid: mda.AtomGroup = res_fixed.select_atoms("resname *")
resid.guess_bonds()
nv.show_mdanalysis(resid)

/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8

NGLWidget()

## 3. Adding New Proton

The carbon on the carboxyl end of the residue only has three bonds when removed from the backbone. Rather than adding a hydroxyl group, a proton is added to the end as a 4th bond. This is done by the `_protonate_backbone()` method in Optimizer.

The new proton confidantes are obtained using the `_get_new_pos()` function by taking the opposite of the sum of the normalized vectors between the $C-\alpha C$ and the $C=O$ bond. That new vector is than normalized, multiplied by the new bond length, and added to the coordinates of the carbon. This gives the trigonal planar geometry expected in a SP2 carbon.

In [10]:
def _get_new_pos(backbone: mda.AtomGroup, length: float):
    c_pos = backbone.select_atoms('name C').positions[0]
    o_pos = backbone.select_atoms('name O').positions[0]
    a_pos = backbone.select_atoms('name CA').positions[0]
    o_pos = o_pos - c_pos  # Translate coords such that C in at origin
    a_pos = a_pos - c_pos
    o_norm = o_pos/np.linalg.norm(o_pos)
    a_norm = a_pos/np.linalg.norm(a_pos)
    h_pos = -(o_norm + a_norm)
    h_norm = h_pos/np.linalg.norm(h_pos)
    h_norm = (h_norm * length) + c_pos
    return h_norm

In [11]:
resid_system = mda.Universe('resid_fixed.pdb') # Load PDBRepair segid
resid_fixed = resid_system.select_atoms('all')
h = mda.Universe.empty(n_atoms=resid_fixed.n_atoms + 1, trajectory=True)
h.add_TopologyAttr("masses", [x for x in resid_fixed.masses] + [1])
h.add_TopologyAttr("name", [x for x in resid_fixed.names] + ['H*'])
resid_pos = resid_fixed.positions
h_pos = _get_new_pos(resid_fixed, 1.128)
h.atoms.positions = np.row_stack((resid_pos, h_pos))
nv.show_mdanalysis(h)


/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/core/universe.py:433: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/core/universe.py:439: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


NGLWidget()

## 4. Optimizing New Bond

Psi4 is employed to optimize the length of the newly added C-H bond. Before this occurs the residue is converted into an rdkit molecule so that the spin multiplicity and formal charge can be easily computed for the psi4 input file.
Afterwards the coordinates are given to psi4 and the geometric optimization is run. This is done during the `_opt_geometry()` method of Optimizer.

In [12]:
from rdkit import Chem

from MDSAPT.mdsapt.optimizer import get_spin_multiplicity

from MDAnalysis.topology.guessers import guess_types, guess_atom_element
from MDAnalysis.converters.RDKit import atomgroup_to_mol

h.add_TopologyAttr('types', guess_types(h.atoms.names))
h.add_TopologyAttr('elements', [guess_atom_element(atom) for atom in h.atoms.names])
resid = h.select_atoms('all')

rd_mol = atomgroup_to_mol(resid)

/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/converters/RDKit.py:419: UserWarning: No `bonds` attribute in this AtomGroup. Guessing bonds based on atoms coordinates
  warnings.warn(


In [13]:
import psi4

coords: str = f'{Chem.GetFormalCharge(rd_mol)} {get_spin_multiplicity(rd_mol)}'
freeze_list = ''
opt_settings: dict = {'reference': 'rhf',
                      'freeze_core': 'true'} # Psi4 settings

# Getting coords and settings
for n in range(len(h.atoms)):
    atom = h.atoms[n]
    coords += f"\n{atom.name[0]} {atom.position[0]} {atom.position[1]} {atom.position[1]}"
    if atom.name != 'H*':
        freeze_list += f"\n{n + 1} xyz" # Freezing bonds besides H
print(freeze_list)
print(coords)

mol = psi4.geometry(coords)



1 xyz
2 xyz
3 xyz
4 xyz
5 xyz
6 xyz
7 xyz
8 xyz
9 xyz
10 xyz
11 xyz
12 xyz
13 xyz
14 xyz
15 xyz
16 xyz
17 xyz
18 xyz
19 xyz
20 xyz
21 xyz
22 xyz
23 xyz
24 xyz
25 xyz
26 xyz
2 1
N 11.840999603271484 6.642000198364258 6.642000198364258
H 10.892999649047852 7.389999866485596 7.389999866485596
H 11.885000228881836 6.519999980926514 6.519999980926514
H 12.565999984741211 7.125 7.125
C 11.414999961853027 5.434000015258789 5.434000015258789
H 10.65999984741211 5.004000186920166 5.004000186920166
C 12.531999588012695 4.363999843597412 4.363999843597412
H 12.118000030517578 3.5220000743865967 3.5220000743865967
H 13.302000045776367 4.796999931335449 4.796999931335449
C 13.315999984741211 3.759000062942505 3.759000062942505
H 12.62399959564209 3.680000066757202 3.680000066757202
H 13.604999542236328 2.6540000438690186 2.6540000438690186
C 14.609000205993652 4.525000095367432 4.525000095367432
H 15.270000457763672 4.505000114440918 4.505000114440918
H 14.281000137329102 5.531000137329102 5.53100

In [14]:
psi4.set_memory('8GB')
psi4.set_options(opt_settings)
psi4.optimize('scf/cc-pvdz', freeze_list=freeze_list, opt_coordinates='cartesian', molecule=mol)


  Memory set to   7.451 GiB by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on Alias-MBP
*** at Tue Dec 28 14:18:28 2021

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 16, 19, 22                                   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-4, 6, 8-9, 11-12, 14-15, 17, 20-21, 23-24, 27 entry H          line    22 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 5, 7, 10, 13, 18, 25                            entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 26                                              entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------

New linear angles


   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 16, 19, 22                                   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-4, 6, 8-9, 11-12, 14-15, 17, 20-21, 23-24, 27 entry H          line    22 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 5, 7, 10, 13, 18, 25                            entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 26                                              entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
               

TORS::compute_val: unable to compute torsion value


OptimizationConvergenceError: Could not converge geometry optimization in 6 iterations.